# 第二章 编写 Prompt 的原则

  本章的主要内容为编写 Prompt 的原则，在本章中，我们将给出两个编写 Prompt 的原则与一些相关的策略，您可以练习编写高效的 Prompt，从而便捷而有效地使用 LLM。

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一环境配置" data-toc-modified-id="一、环境配置">一、环境配置</a></span></li>
        <li>
            <span><a href="#二两个基本原则" data-toc-modified-id="二、两个基本原则">二、两个基本原则</a></span>
            <ul class="toc-item">
                <li><span><a href="#21-原则一编写清晰具体的指令" data-toc-modified-id="2.1 原则一：编写清晰、具体的指令">2.1 原则一：编写清晰、具体的指令</a></span></li>
                <li><span><a href="#22-给模型时间去思考" data-toc-modified-id="2.2 原则二：给模型时间去思考">2.2 原则二：给模型时间去思考</a></span></li>
            </ul>
        </li>
        <li><span><a href="#三局限性" data-toc-modified-id="三、局限性">三、局限性</a></span>
        </li>
    </ul>
</div>

## 一、环境配置

本教程使用 OpenAI 所开放的 ChatGPT API，因此您需要首先拥有一个 ChatGPT 的 API_KEY（也可以直接访问官方网址在线测试），然后需要安装 OpenAI 的第三方库。为了兼顾简便与兼容性，本教程将介绍在 ```Python 3``` 环境中基于 ```openai.api_key``` 方法的配置。另有基于环境变量的配置方法，详情请参考 [OpenAI 官方文档](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)。

首先需要安装 OpenAI 库：
```bash
pip install openai
```

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

整个课程将以 gpt-3.5-turbo 模型<span style="color:#00ff00">（我将使用gpt-4o-mini模型）</span>为例。我们将在后续课程中深入探究 OpenAI 提供的 [Chat Completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api) 的使用方法，在此处，我们先将它封装成一个函数，您无需知道其内部机理，仅需知道调用该函数，以 Prompt 为输入参数，其将会输出对应的 Completion （回答结果）即可。

In [2]:
# 初始化客户端
client = OpenAI(api_key=openai_api_key)

# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_openai_completion(prompt, model="gpt-4o-mini"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为gpt-4o-mini
    '''
    # 调用API
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,  # 模型输出的温度系数，控制输出的随机程度
    )
    # 返回结果
    return response.choices[0].message.content

<span style="color:#00ff00">
也可以使用deepseek的api接口作为替代
</span>

In [3]:
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

client_ds = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")

def get_deepseek_completion(prompt, model="deepseek-chat"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为deepseek-v3
    '''
    # 调用API
    response = client_ds.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    # 返回结果
    return response.choices[0].message.content

## 二、两个基本原则

### 2.1 原则一：编写清晰、具体的指令

您应该通过提供<span style="color:red">尽可能清晰和具体的指令</span>来表达您希望模型执行的操作。这将引导模型给出正确的输出，并降低您得到无关或不正确响应的可能性。清晰的指令不意味着必须简短，在许多情况下，更长的 Prompt 实际上更清晰，且提供了更多上下文，也就可能产生更详细更相关的输出。

**2.1.1 使用<span style="color:red">分隔符</span>
清晰地表示输入的不同部分**

分隔符可以是：```，""，<>，:，\<tag> \</tag>等。

您可以使用任何明显的标点符号将特定的文本部分与 Prompt 的其余部分分开。标记的形式不限，只需要让模型明确知道这是一个单独部分。使用分隔符可以有效避免提示词注入( Prompt injection )。提示词注入是指如果允许用户将某些输入添加到（开发者预定义的） Prompt 中，则所提供的指令可能会与开发者想要执行的操作相冲突，从而使 LLM 遵循用户输入的指令，而非执行开发者预期的操作。即，输入里面可能包含其他指令，会覆盖掉您的指令。对此，使用分隔符是一个不错的策略。

在以下的例子中，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符。


In [4]:
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_openai_completion(prompt)
print(f"LLM: {response}")

LLM: 提供清晰、具体且详细的指示可以帮助模型更准确地执行任务并减少无关响应的可能性。


<span style="color:#00ff00">
对于中文内容，deepseek-v3模型表现比gpt-4o-mini更好。但是deepseek-v3模型的响应速度不如gpt-4o-mini。
</span>

In [5]:
response_ds = get_deepseek_completion(prompt)
print(f"LLM: {response_ds}")

LLM: 提供清晰、具体的指示有助于引导模型生成更相关和准确的输出，而更长的提示词通常能提供更多上下文信息，从而提高输出的质量。


**2.1.2 寻求<span style="color:red">结构化</span>的输出**

输出可以是 Json、HTML 等格式。

第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，您可以在 Python 中将其读入字典或列表中。

在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [6]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_deepseek_completion(prompt)
print(response)

以下是三本虚构书籍的清单，以 JSON 格式提供：

```json
[
    {
        "book_id": 1,
        "title": "星辰之海的秘密",
        "author": "艾琳娜·莫里斯",
        "genre": "科幻"
    },
    {
        "book_id": 2,
        "title": "迷雾中的回声",
        "author": "杰克·哈珀",
        "genre": "悬疑"
    },
    {
        "book_id": 3,
        "title": "永恒花园的守护者",
        "author": "莉莉安·格雷",
        "genre": "奇幻"
    }
]
```

这个清单包含了三本虚构书籍，每本书都有唯一的 `book_id`、书名 (`title`)、作者 (`author`) 和类别 (`genre`)。


**2.1.3 要求模型检查是否满足条件**

如果任务包含不一定能满足的假设（条件），我们可以告诉模型先检查这些假设，如果不满足，则会指出并停止执行后续的完整流程。您还可以考虑可能出现的边缘情况及模型的应对，以避免意外的结果或错误发生。

在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答“未提供步骤”。

In [7]:
# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个尖括号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
<<<{text_1}>>>
"""
response = get_deepseek_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开。  
第二步 - 拿一个杯子并把茶包放进去。  
第三步 - 把烧开的水倒在茶包上。  
第四步 - 等待几分钟，让茶叶浸泡。  
第五步 - 取出茶包。  
第六步 - 根据个人喜好，加一些糖或牛奶调味。  
第七步 - 享受一杯美味的茶。


In [8]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个尖括号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
<<<{text_2}>>>
"""
response = get_deepseek_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤


**2.1.4 提供少量示例**（少样本提示词，Few-shot prompting）

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和祖父之间的对话的例子。孩子说，“请教我何为耐心”，祖父用下述风格的隐喻来回答。由于我们已经告诉模型要以一致的语气回答，因此现在我们问“请教我何为韧性”，由于模型已经有了这个<span style="color:red">少样本示例( few-shot example )</span>，它将以类似的语气回答下一个任务。

In [36]:
prompt = f"""
您的任务是以一致的风格扮演祖父回答问题。

<孩子>: 请教我何为耐心。

<祖父>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 请教我何为韧性。
"""
response = get_deepseek_completion(prompt)
print(response)

<祖父>: 柳枝在暴风中低吟却不断裂，是因懂得弯曲而非抵抗；精钢历经千次淬火方能斩断磐石，是因甘愿承受锤击而非退缩；寒冬的大地默默承受冰封，却深埋着破土而出的嫩芽。孩子啊，真正的韧性不在于永不跌倒，而在于每次跪倒时，掌心始终攥着站起来的星光。


### 2.2 给模型时间去思考

如果您发现模型推理过程过于匆忙，导致得出了错误的结论，那么您应该尝试重新构思 Prompt ，要求模型在提供最终答案之前开展<span style="color:red">思维链</span>，或进行一系列相关推理（a chain or series of relevant reasoning）。换句话说，如果您给模型一个在短时间内或用少量文字无法完成的复杂任务，它的输出结果就容易出错。这种情况对人来说也是类似：如果您要求某人完成复杂的数学问题，又不给足够时间计算出答案，他们也可能会犯错误。因此，在这些情况下，您应该指示模型花更多时间思考问题，让它在任务上花费更多计算资源。

**2.2.1 指定完成任务所需的<span style="color:red">步骤</span>**

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果。

首先我们描述了杰克和吉尔的故事，并给出提示词执行以下操作：首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和人名个数。要求输出以换行符分隔。

In [37]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_deepseek_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
兄妹杰克和吉尔在打水时遭遇意外，但安全回家并保持冒险精神。

Jack et Jill, un frère et une sœur, rencontrent un accident en allant chercher de l'eau à un puits au sommet d'une colline mais rentrent chez eux en conservant leur esprit aventurier.

Noms: Jack, Jill

{
  "French_summary": "Jack et Jill, un frère et une sœur, rencontrent un accident en allant chercher de l'eau à un puits au sommet d'une colline mais rentrent chez eux en conservant leur esprit aventurier.",
  "num_names": 2
}


上述输出仍然存在一定问题，例如，键“姓名”会被替换为法语（译注：在英文原版中，对应指令第三步的输出为 'Noms:',为Name的法语，这种行为难以预测，并可能为导出带来困难）

因此，我们将Prompt加以改进，该 Prompt 前半部分不变，同时**确切指定了输出的格式**。

In [42]:
prompt_2 = f"""
请执行以下操作：
1-用一句话概括下面用三个尖括号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请使用以下格式：
摘要：<摘要>
翻译：<摘要的翻译>
名称：<法语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text:
<<<{text}>>>
"""
response = get_deepseek_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要：杰克和吉尔兄妹在村庄附近的山顶打水时遭遇意外滚落山下，但最终安全返家并保持了冒险精神。  
翻译：Dans un village charmant, les frère et sœur Jack et Jill partent chercher de l'eau à un puits au sommet d'une colline, tombent accidentellement en descendant, mais rentrent chez eux en conservant leur esprit d'aventure.  
名称：["Jack", "Jill"]  
输出 JSON：  
```json
{
  "French_summary": "Dans un village charmant, les frère et sœur Jack et Jill partent chercher de l'eau à un puits au sommet d'une colline, tombent accidentellement en descendant, mais rentrent chez eux en conservant leur esprit d'aventure.",
  "num_names": 2
}
```


**2.2.2 指导模型在下结论之前找出一个自己的解法**

明确地指引模型在匆匆做决策之前，要<span style="color:red">自己思考出一份解决方案</span>。有时这样会得到更好的结果。这与之前所述思想类似，即给模型时间思考。

接下来我们会给出一个问题和一份来自学生的解答，要求模型判断解答是否正确：

<span style="color:#00ff00">
deepseek-v3模型目前已经基本能够自我判断学生的解答，不需要额外的提示词。
</span>

In [10]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_deepseek_completion(prompt)
print(response)

学生的解决方案是正确的。

让我们逐步分析学生的计算过程：

1. **土地费用**：每平方英尺的土地费用是100美元，因此土地费用为 \(100x\) 美元。

2. **太阳能电池板费用**：每平方英尺的太阳能电池板费用是250美元，因此太阳能电池板费用为 \(250x\) 美元。

3. **维护费用**：维护费用包括固定费用和可变费用。固定费用是每年10万美元，可变费用是每平方英尺10美元，因此维护费用为 \(100,000 + 10x\) 美元。

4. **总费用**：将上述所有费用相加，得到总费用：
   \[
   100x + 250x + 100,000 + 10x = 360x + 100,000
   \]
   然而，学生在计算总费用时写的是 \(450x + 100,000\) 美元，这里有一个小错误。实际上，总费用应该是 \(360x + 100,000\) 美元。

**修正后的总费用**：
\[
\text{总费用} = 100x + 250x + 10x + 100,000 = 360x + 100,000
\]

因此，学生的解决方案在计算总费用时有一个小错误，正确的总费用应该是 \(360x + 100,000\) 美元。


但是注意，学生的解决方案实际上是错误的。（*维护费用项100x应为10x，总费用450x应为360x*）

我们可以通过指导模型先自行找出一个解法来解决这个问题。

在接下来这个 Prompt 中，我们要求模型先自行解决这个问题，再根据自己的解法与学生的解法进行对比，从而判断学生的解法是否正确。同时，我们给定了输出的格式要求。通过拆分任务、明确步骤，让模型有更多时间思考，有时可以获得更准确的结果。在这个例子中，学生的答案是错误的，但如果我们没有先让模型自己计算，那么可能会被误导以为学生是正确的。

In [11]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：
首先，自己解决问题，解决问题时列数学表达式。
然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
学生的计算结果：学生的计算结果文本
实际计算结果：实际计算结果文本
学生的计算结果和实际计算结果是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否
学生的成绩：正确或不正确

以下是需要判断的问题和解决方案，请按照上述格式补全剩余内容。

问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_deepseek_completion(prompt)
print(response)

问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+10x
总费用：100x+250x+100,000+10x=360x+100,000

学生的计算结果：450x+100,000
实际计算结果：360x+100,000
学生的计算结果和实际计算结果是否相同：否
学生的解决方案和实际解决方案是否相同：否
学生的成绩：不正确


## 三、局限性

**开发大模型相关应用时请务必铭记：**


**虚假知识**：模型偶尔会生成一些看似真实实则编造的知识

虽然模型在训练过程中接触了大量的知识，但它并没有*完全*记住所见的信息，因此它不甚清楚自己知识的边界。这意味着它可能会尝试回答主题晦涩难懂的问题，并编造听起来合理但实际上并不正确的答案。我们称这些编造的想法为<span style="color:red">幻觉（Hallucination）</span>。

如下示例展示了大模型的幻觉。我们要求告诉我们 Boie 公司生产的 *AeroGlide UltraSlim Smart Toothbrush* 产品的信息，事实上，这个公司是真实存在的，但产品是编造的，而模型一本正经地提供了它编造的知识，而且迷惑性很强。



<span style="color:#00ff00">
更强的大模型在此事例中不会表现出幻觉，但其幻觉现象仍然存在。
</span>

In [12]:
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息
"""
response = get_deepseek_completion(prompt)
print(response)

截至2023年10月，我无法找到关于 Boie 公司生产的 **AeroGlide UltraSlim Smart Toothbrush** 的具体信息。这可能是因为该产品并不存在，或者相关信息尚未广泛传播。以下是一些可能相关的背景信息和建议：

1. **Boie 公司**：Boie 是一家以设计和生产创新个人护理产品而闻名的公司，尤其以其环保牙刷和身体护理产品著称。他们的牙刷通常采用可回收材料制成，设计简约且注重可持续性。

2. **智能牙刷**：如果 Boie 推出了一款智能牙刷，它可能会具备以下功能：
   - 蓝牙连接，可通过手机应用程序跟踪刷牙习惯。
   - 压力传感器，防止过度用力刷牙。
   - 定时功能，确保用户刷牙时间达到推荐的两分钟。
   - 可更换刷头，延续 Boie 的环保理念。

3. **AeroGlide UltraSlim**：如果这是 Boie 的一款产品，它可能以超薄设计为卖点，适合旅行或随身携带，同时保持高效清洁能力。

4. **建议**：
   - 如果您对 Boie 的产品感兴趣，可以访问其官方网站或联系客服以获取最新信息。
   - 如果您在寻找智能牙刷，可以考虑其他知名品牌，如 **Oral-B**、**Philips Sonicare** 或 **Quip**。

如果您有更多具体信息或问题，请告诉我，我会尽力帮助！


由于很容易以假乱真，请读者根据在本系列教程中所学知识，在构建自己的应用程序时尽量避免幻觉情况。幻觉是大模型的一个已知缺陷（注：截至2023年7月），OpenAI也在努力解决该问题。

在您希望模型根据文本生成回答时，另一种减少幻觉的策略是先要求模型获取来源于该文本的所有引用信息（任何相关引用，any relevant quotes），然后要求它基于所引用的信息来回答问题，这使得我们能根据答案追溯源文档，通常对减少幻觉非常有帮助。

**关于反斜杠使用的说明：**

在本教程中，我们使用反斜杠 \ 来使文本适应屏幕大小以提高阅读体验，而没有用换行符 \n 。GPT-3 并不受换行符（newline characters）的影响，但在您调用其他大模型时，需额外考虑换行符是否会影响模型性能。